In [32]:
import sys
import os

import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
# from tqdm import tqdm
from tqdm.notebook import tqdm

import tushare as ts
from utils import plot_k

sys.path.insert(0, './../tushare_code/')

In [33]:
#stock_lists = ['000651', '600000', '000061', '601398', '600050']
stock_lists = ['600050']
data_path = '../data/stockData/allstock/'
new_names = {
    '股票代码': 'stock',
    '交易日期':  'date',
    '开盘价'  : 'open',
    '最高价'  : 'high',
    '最低价'  : 'low',
    '收盘价'  : 'close',
}
# cols = ['股票代码','交易日期','开盘价','最高价','最低价','收盘价']
cols = [0, 1, 2, 3, 4, 5]
names = ['stock', 'date', 'open', 'high', 'low', 'close']

In [34]:
def get_dark_cloud_indices(df):
    res = []
    df_close = df.close
    df_open = df.open
    df_diff = df.close - df.open
    stats = df_diff.describe() # count, mean, std, min, 25%, 50%, 75%, max 
    lag1 = df_diff.shift(1)
    
    lag_open = df_open.shift(1)
    lag_close = df_close.shift(1)
    # 捕捉红色实体，和绿色实体
    shape = [0, 0, 0]
    for i in range(3, len(df_diff)):
        if all([lag1[i] > 0, lag1[i] > abs(stats[5]), df_diff[i] < 0, df_open[i] > lag_close[i],
               df_close[i] < (lag_close[i] + lag_open[i]) / 2]):
            shape.append(1)
        else:
            shape.append(0)
            
    # 定义上升趋势
    trend = [0, 0, 0]
    for i in range(3, len(df_close)):
        if all([lag_close[i] > lag_close[i-1], lag_close[i-1] > lag_close[i-2]]):
            trend.append(1)
        else:
            trend.append(0)
    
    star_sig = []
    for i in range(len(trend)):
        if all([shape[i]==1, trend[i]==1]):
            star_sig = 1
            res.append(df.index[i])
        else:
            star_sig = 0
    return res

In [35]:
def get_price_diff(df, dates, lag=7):
    inc = []
    dec = []
    for d in dates:
        next_d = (df.index == d).tolist().index(True) + lag
        diff =  df.iloc[next_d].close / df.loc[d].close - 1
        if diff > 0:
            inc.append(d)
        else:
            dec.append(d)
    return inc, dec    

In [23]:
# df = ts.get_k_data('600050', start='2000-01-01', end='2020-01-01')
# df = df.set_index(pd.to_datetime(df.date))
# df = df.drop(['date'], axis=1)
# indices = get_dark_cloud_indices(df)

In [36]:
#len(indices)

In [25]:
# plot_k(df, start='2019-11-01', end='2019-12-01')
#plot_k(df, start='2000-03-21', end='2000-05-21')

In [29]:
# for stock in stock_lists:
#     df = ts.get_k_data(stock, start='2000-01-01', end='2020-01-01')
#     df = df.set_index(pd.to_datetime(df.date))
#     df = df.drop(['date'], axis=1)
#     indices = get_dark_cloud_indices(df)
#     inc, dec = get_price_diff(df, indices, lag=7)
#     print('Stock {}, Price increase times {}'.format(stock, len(inc)))
#     print('Stock {}, Price decrease times {}'.format(stock, len(dec)))
#     if len(dec):
#         print('ratio: {}'.format(len(inc) / len(dec)))

Stock 000651, Price increase times 58
Stock 000651, Price decrease times 41
ratio: 1.4146341463414633


In [38]:
files = os.listdir(data_path)
print("There are total {} stocks".format(len(files)))

There are total 3905 stocks


In [39]:
total_inc = 0
total_dec = 0
for f in tqdm(files):
    if f.endswith('.xls'):
        #print('Processing file {}'.format(f))
        inc = []
        dec = []
        try:
            data_f = os.path.join(data_path, f)
            df = pd.read_excel(data_f, usecols=cols, names=names)
            df = df.set_index(pd.to_datetime(df["date"], format='%Y%m%d'))
            df = df.drop(['date'], axis=1)
            df = df[::-1]
            indices = get_dark_cloud_indices(df)
            inc, dec = get_price_diff(df, indices, lag=7)
    #         print('There are  {} records for stock {}'.format(len(df), f))
    #         print('Stock {}, Price increase times {}'.format(f, len(inc)))
    #         print('Stock {}, Price decrease times {}'.format(f, len(dec)))
    #         if len(dec):
    #             print('ratio: {}'.format(len(inc) / len(dec)))
    #         print('---------------')
        except:
            print('An error occurred for file {}'.format(f))
    total_inc += len(inc)
    total_dec += len(dec)
print('total inc {}'.format(total_inc))
print('total dec {}'.format(total_dec))

  0%|          | 0/3905 [00:00<?, ?it/s]

An error occurred for file 603730.SH.xls
An error occurred for file 603587.SH.xls
An error occurred for file 600701.SH.xls
An error occurred for file 600150.SH.xls
An error occurred for file 300121.SZ.xls
An error occurred for file 300021.SZ.xls
An error occurred for file 603499.SH.xls
An error occurred for file 000880.SZ.xls
An error occurred for file 600284.SH.xls
An error occurred for file 002675.SZ.xls
An error occurred for file 002779.SZ.xls
An error occurred for file 300601.SZ.xls
An error occurred for file 002767.SZ.xls
An error occurred for file 300729.SZ.xls
An error occurred for file 300178.SZ.xls
An error occurred for file 600658.SH.xls
An error occurred for file 603038.SH.xls
An error occurred for file 603677.SH.xls
An error occurred for file 002935.SZ.xls
An error occurred for file 600580.SH.xls
An error occurred for file 600480.SH.xls
An error occurred for file 002489.SZ.xls
An error occurred for file 300592.SZ.xls
An error occurred for file 603970.SH.xls
An error occurre

An error occurred for file 000969.SZ.xls
An error occurred for file 002207.SZ.xls
An error occurred for file 300273.SZ.xls
An error occurred for file 601138.SH.xls
An error occurred for file 002234.SZ.xls
An error occurred for file 002518.SZ.xls
An error occurred for file 601901.SH.xls
An error occurred for file 002418.SZ.xls
An error occurred for file 002349.SZ.xls
An error occurred for file 002326.SZ.xls
An error occurred for file 000720.SZ.xls
An error occurred for file 600352.SH.xls
An error occurred for file 600252.SH.xls
An error occurred for file 002414.SZ.xls
An error occurred for file 601618.SH.xls
An error occurred for file 600889.SH.xls
An error occurred for file 002541.SZ.xls
An error occurred for file 300790.SZ.xls
An error occurred for file 600997.SH.xls
An error occurred for file 002880.SZ.xls
An error occurred for file 000059.SZ.xls
An error occurred for file 002373.SZ.xls
An error occurred for file 002795.SZ.xls
An error occurred for file 300219.SZ.xls
An error occurre

An error occurred for file 600835.SH.xls
An error occurred for file 600497.SH.xls
total inc 85138
total dec 83984
